In [1]:
import sys

print(sys.executable)

/home/boom/.pyenv/versions/aivet/bin/python


In [2]:
import os
import torch
import fitz  # PyMuPDF
import io
from typing import List, Dict, Any, Optional
from PIL import Image

# LangChain imports
from langchain_community.chat_models import ChatDeepInfra
from langchain_ollama import ChatOllama
from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.schema import Document

# HuggingFace imports for vision model
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# /home/boom/.pyenv/versions/aivet/bin/python -m pip install --upgrade pip
# /home/boom/.pyenv/versions/aivet/bin/python -m pip install -r requirements.txt

In [4]:
import ollama

ollama_client = ollama.Client()
ollama_client.list()["models"]

available_models = [model["model"] for model in ollama_client.list()["models"]]
available_models

['qwen2.5vl:7b',
 'llama3.2-vision:11b',
 'gemma3:12b-it-qat',
 'qwen2.5:7b',
 'gemma3:4b',
 'qwen3:8b',
 'qwen3:0.6b',
 'gemma3:27b-it-qat',
 'deepseek-r1:7b',
 'qwen2.5:14b-instruct-q4_K_M']

In [5]:
# ollama_client.pull(f"gemma3:12b-it-qat")

In [6]:
class ModelProvider:
    """Centralized model provider management for different LLM services."""

    def _load_ollama_model(self, model_name):
        """Initialize Ollama client for multimodal models."""
        try:
            import ollama

            self.ollama_client = ollama.Client()

            # Verify model exists, pull if needed
            available_models = [
                model["model"] for model in self.ollama_client.list()["models"]
            ]
            if model_name not in available_models:
                print(f"📥 Pulling Ollama model: {model_name}")
                self.ollama_client.pull(f"{model_name}")

            print(f"✅ Ollama multimodal model {model_name} ready")
        except Exception as e:
            print(f"❌ Error initializing Ollama model: {e}")
            raise

    def initialize_model(
        self, provider: str = "ollama", model_name="qwen2.5:8b", **kwargs
    ) -> Any:
        """Initialize the appropriate model based on provider."""
        provider = provider.lower()

        print(provider)
        print(model_name)
        if provider == "ollama":
            self._load_ollama_model(model_name)
            if "qwen2.5" in model_name:
                return ChatOllama(
                    model=model_name,
                    temperature=kwargs.get("temperature", 0.6),
                    top_p=kwargs.get("top_p", 0.95),
                    num_ctx=kwargs.get("num_ctx", 32768),
                )
            elif "gemma3" in model_name:
                return ChatOllama(
                    model="gemma3:8b",
                    temperature=kwargs.get("temperature", 1.0),
                    top_k=kwargs.get("top_k", 64),
                    top_p=kwargs.get("top_p", 0.95),
                    num_ctx=kwargs.get("num_ctx", 32768),
                )
        elif provider == "anthropic":

            return ChatLiteLLM(
                model="anthropic/claude-3-5-haiku-latest",
                api_key=os.getenv("ANTHROPIC_API_KEY"),
                temperature=kwargs.get("temperature", 0.6),
                max_tokens=kwargs.get("max_tokens", 8192),
            )
        elif provider == "deepinfra":
            return ChatDeepInfra(
                model_name="Qwen/Qwen2.5-72B-Instruct",
                deepinfra_api_token=os.getenv("DEEPINFRA_API_KEY"),
                max_tokens=kwargs.get("max_tokens", 8192),
                temperature=kwargs.get("temperature", 0.6),
                top_p=kwargs.get("top_p", 0.95),
                top_k=kwargs.get("top_k", 20),
            )
        elif provider == "meta":
            return ChatDeepInfra(
                model_name="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                deepinfra_api_token=os.getenv("DEEPINFRA_API_KEY"),
                max_tokens=kwargs.get("max_tokens", 8192),
                temperature=kwargs.get("temperature", 0.7),
                streaming=kwargs.get("streaming", True),
            )
        elif provider == "deepseek":
            return ChatDeepInfra(
                model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
                api_base="https://api.deepinfra.com/v1/openai",
                api_key=os.getenv("DEEPINFRA_API_KEY"),
                temperature=kwargs.get("temperature", 0.6),
                top_p=kwargs.get("top_p", 0.95),
            )
        elif provider == "local_llama":
            return LlamaCpp(
                model_name=kwargs.get("model_name", ""),
                n_ctx=kwargs.get("n_ctx", 5000),
                n_threads=kwargs.get("n_threads", 12),
                n_gpu_layers=kwargs.get("n_gpu_layers", 0),
                temperature=kwargs.get("temperature", 0.7),
                top_p=kwargs.get("top_p", 0.95),
                verbose=False,
            )
        else:
            raise ValueError(f"❌ Unsupported provider: {provider}")


class ImageToTextExtractor:
    """Modernized image-to-text extractor supporting both HuggingFace and Ollama multimodal models."""

    def __init__(
        self,
        vision_model_name: str = None,
        vision_model_provider: str = "huggingface",
        device: str = "auto",
        **kwargs,
    ):
        self.device = (
            device
            if device != "auto"
            else ("cuda" if torch.cuda.is_available() else "cpu")
        )
        self.model_name = vision_model_name
        self.model_provider = vision_model_provider.lower()
        self.model = None
        self.tokenizer = None
        self.ollama_client = None

        # Additional kwargs for model configuration
        self.model_config = kwargs

        self._load_model()

        self.extraction_query = """Task: Extract and list all blood test details from this veterinary blood test report.

        Objective:
        1. Identify and extract **chemical/analyte names** from the blood test report.
        2. Capture the **detected levels** of each chemical with their units.
        3. Extract the **normal/reference range** for each corresponding chemical.
        4. Note any flags or indicators (High, Low, Normal, etc.).

        Expected Output Format:
        - Chemical Name: [Name]
        - Detected Level: [Value with units]
        - Normal Range: [Min Value] - [Max Value with units]
        - Status: [Normal/High/Low if indicated]

        Please ensure accurate extraction, including any unit symbols (e.g., mg/dL, IU/L, g/L), and handle variations in formatting or alignment. Return results in a structured list format for easy readability."""

    def _load_model(self):
        """Load the appropriate vision model based on provider."""
        if self.model_provider == "huggingface":
            self._load_huggingface_model()
        elif self.model_provider == "ollama":
            self._load_ollama_model()
        else:
            raise ValueError(f"Unsupported model provider: {self.model_provider}")

    def _load_huggingface_model(self):
        """Load HuggingFace vision model."""
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                low_cpu_mem_usage=True,
                trust_remote_code=True,
                device_map="auto",
                use_cache=True,
                **self.model_config,
            )
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name, trust_remote_code=True
            )
            print(f"✅ HuggingFace vision model loaded successfully on {self.device}")
        except Exception as e:
            print(f"❌ Error loading HuggingFace vision model: {e}")
            raise

    def _load_ollama_model(self):
        """Initialize Ollama client for multimodal models."""
        try:
            import ollama

            self.ollama_client = ollama.Client()

            # Verify model exists, pull if needed
            available_models = [
                model["model"] for model in self.ollama_client.list()["models"]
            ]
            if self.model_name not in available_models:
                print(f"📥 Pulling Ollama model: {self.model_name}")
                self.ollama_client.pull(f"{self.model_name}")

            print(f"✅ Ollama multimodal model {self.model_name} ready")
        except ImportError:
            raise ImportError(
                "Ollama package not found. Install with: pip install ollama"
            )
        except Exception as e:
            print(f"❌ Error initializing Ollama model: {e}")
            raise

    def clear_memory(self):
        """Clear GPU memory based on provider."""
        if self.model_provider == "huggingface":
            if self.model is not None:
                del self.model
                del self.tokenizer
                self.model = None
                self.tokenizer = None
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        elif self.model_provider == "ollama":
            # Ollama manages its own memory, but we can clear the client reference
            self.ollama_client = None
        print("🧹 Memory cleared")

    def convert_pdf_to_images(
        self, pdf_path: str, zoom: float = 2.0
    ) -> List[Image.Image]:
        """Convert PDF pages to PIL Images."""
        images = []
        mat = fitz.Matrix(zoom, zoom)

        try:
            doc = fitz.open(pdf_path)
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                pix = page.get_pixmap(matrix=mat)
                img_data = pix.tobytes("png")
                img = Image.open(io.BytesIO(img_data))
                images.append(img.convert("RGB"))
                print(f"📄 Converted page {page_num + 1} to image")
            doc.close()
            return images
        except Exception as e:
            print(f"❌ Error converting PDF: {e}")
            raise

    def extract_text_from_file(self, file_path: str) -> str:
        """Extract text from PDF or image file."""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

        file_extension = os.path.splitext(file_path)[1].lower()
        extracted_text = ""

        try:
            if file_extension == ".pdf":
                print(f"📋 Processing PDF file: {file_path}")
                images = self.convert_pdf_to_images(file_path)
                for i, image in enumerate(images):
                    print(f"🔍 Extracting text from page {i + 1}...")
                    page_text = self._process_image(image, self.extraction_query)
                    extracted_text += f"=== Page {i + 1} ===\n{page_text}\n\n"
            elif file_extension in [".png", ".jpeg", ".jpg", ".bmp"]:
                print(f"🖼️ Processing image file: {file_path}")
                image = Image.open(file_path).convert("RGB")
                extracted_text = self._process_image(image, self.extraction_query)
            else:
                raise ValueError(f"Unsupported file type: {file_extension}")

            return extracted_text.strip()
        except Exception as e:
            print(f"❌ Error during text extraction: {e}")
            raise

    def _process_image(self, image: Image.Image, query: str) -> str:
        """Process a single image with the appropriate vision model."""
        if self.model_provider == "huggingface":
            return self._process_image_huggingface(image, query)
        elif self.model_provider == "ollama":
            return self._process_image_ollama(image, query)
        else:
            raise ValueError(f"Unsupported model provider: {self.model_provider}")

    def _process_image_huggingface(self, image: Image.Image, query: str) -> str:
        """Process image using HuggingFace model."""
        if self.model is None or self.tokenizer is None:
            raise RuntimeError(
                "HuggingFace model not loaded. Call _load_model() first."
            )

        try:
            # Prepare inputs
            inputs = self.tokenizer.apply_chat_template(
                [{"role": "user", "image": image, "content": query}],
                add_generation_prompt=True,
                tokenize=True,
                return_tensors="pt",
                return_dict=True,
            )

            inputs = inputs.to(self.device)

            gen_kwargs = {
                "max_length": 2500,
                "do_sample": True,
                "top_k": 1,
                "temperature": 0.2,
                "pad_token_id": self.tokenizer.eos_token_id,
            }

            with torch.no_grad():
                outputs = self.model.generate(**inputs, **gen_kwargs)
                outputs = outputs[:, inputs["input_ids"].shape[1] :]
                output_text = self.tokenizer.decode(
                    outputs[0], skip_special_tokens=True
                )
                return output_text.strip()
        except Exception as e:
            print(f"❌ Error processing image with HuggingFace: {e}")
            return ""

    def _process_image_ollama(self, image: Image.Image, query: str) -> str:
        """Process image using Ollama multimodal model."""
        if self.ollama_client is None:
            raise RuntimeError(
                "Ollama client not initialized. Call _load_model() first."
            )

        try:
            import base64
            from io import BytesIO

            # Convert PIL image to base64
            buffer = BytesIO()
            image.save(buffer, format="PNG")
            img_base64 = base64.b64encode(buffer.getvalue()).decode()

            # Make request to Ollama
            response = self.ollama_client.generate(
                model=self.model_name,
                prompt=query,
                images=[img_base64],
                options={
                    "temperature": self.model_config.get("temperature", 0.2),
                    "top_k": self.model_config.get("top_k", 1),
                    "top_p": self.model_config.get("top_p", 0.9),
                    "num_ctx": self.model_config.get(
                        "num_ctx", 128000
                    ),  # Gemma 3 supports 128K context
                },
            )

            return response["response"].strip()
        except Exception as e:
            print(f"❌ Error processing image with Ollama: {e}")
            return ""


import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

import os
import shutil
from typing import List, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader, UnstructuredPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
import chromadb
from chromadb.config import Settings

import os
import shutil
import torch
from typing import List, Optional
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import shutil
import torch
from typing import List, Optional
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

class VeterinaryRAG:
    def __init__(self, persist_directory: str = "./chroma_db"):
        self.persist_directory = persist_directory
        self.vectorstore = Chroma(
            embedding=self._setup_embeddings(), persist_directory=self.persist_directory
        )
        self.retriever = self.vectorstore.as_retriever(
            search_type="similarity", search_kwargs={"k": 5}
        )
        print("📚 VeterinaryRAG loaded vector store")

    def _setup_embeddings(self):
        return HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
        )


class VeterinaryTextAnalyzer:
    """Modernized text analyzer with LangChain integration and RAG support."""

    def __init__(
        self,
        llm_provider: str = "ollama",
        llm_model_name: str = "qwen2.5:7b",
        knowledge_base_path: Optional[str] = None,
        **llm_kwargs,
    ):
        print("1", llm_provider)
        self.llm_provider = llm_provider
        provider_instance = ModelProvider()

        self.llm = provider_instance.initialize_model(
            provider=llm_provider, model_name=llm_model_name, **llm_kwargs
        )
        self.rag_system = None

        if knowledge_base_path and os.path.exists(knowledge_base_path):
            self.rag_system = VeterinaryRAG(knowledge_base_path)
            print("✅ RAG system initialized")

        self._setup_prompts()
        self._setup_chains()

    def _setup_prompts(self):
        """Setup prompt templates for different analysis modes."""

        # Chain of Thought prompt
        self.cot_prompt = ChatPromptTemplate.from_template(
            """
        You are a veterinary assistant analyzing blood test results for a dog. Follow these steps:

        Step 1: Review the blood test results below.
        Step 2: For each molecule, compare detected level with normal range and determine if abnormal.
        Step 3: For abnormal values, explain the clinical significance.
        Step 4: Provide a comprehensive health assessment.

        {context}

        Blood test results:
        {blood_results}

        Please provide your detailed step-by-step analysis:
        """
        )

        # Few-shot prompt
        self.few_shot_prompt = ChatPromptTemplate.from_template(
            """
        You are a veterinary assistant analyzing blood test results. Identify abnormal values and their implications.

        Example Analysis:
        Blood results: AST: 50 (Normal: 10-45), ALT: 25 (Normal: 10-60), ALP: 200 (Normal: 45-152)
        
        Analysis:
        - AST: High (50, normal 10-45) - possible liver/muscle damage
        - ALP: High (200, normal 45-152) - possible liver disease or bone disorders
        
        Summary: Elevated liver enzymes suggest hepatic dysfunction requiring further investigation.

        ---

        Veterinary Context:
        {context}

        Now analyze these results:
        {blood_results}
        """
        )

        # Summary prompt
        self.summary_prompt = ChatPromptTemplate.from_template(
            """
        Based on the veterinary knowledge provided and the blood test analysis, provide a concise health summary.

        Veterinary Knowledge:
        {context}

        Blood Test Analysis:
        {analysis_results}

        Provide a focused summary covering:
        1. Key abnormalities and their clinical significance
        2. Potential diagnoses or conditions
        3. Recommended next steps (if warranted)

        Keep response to 5 sentences maximum.
        """
        )

    def _setup_chains(self):
        """Setup LangChain processing chains."""
        self.cot_chain = self.cot_prompt | self.llm | StrOutputParser()
        self.few_shot_chain = self.few_shot_prompt | self.llm | StrOutputParser()
        self.summary_chain = self.summary_prompt | self.llm | StrOutputParser()

    def _get_context(self, query: str) -> str:
        """Get relevant context from RAG system if available."""
        if self.rag_system:
            return self.rag_system.retrieve_context(query)
        return "No additional veterinary knowledge available."

    def analyze_blood_results(self, blood_results: str, mode: str = "few_shot") -> str:
        """Analyze blood test results using specified mode."""
        context = self._get_context(f"blood test analysis {blood_results}")

        try:
            if mode == "chain_of_thought":
                result = self.cot_chain.invoke(
                    {"context": context, "blood_results": blood_results}
                )
            elif mode == "few_shot":
                result = self.few_shot_chain.invoke(
                    {"context": context, "blood_results": blood_results}
                )
            else:
                raise ValueError(f"Unsupported analysis mode: {mode}")

            return result
        except Exception as e:
            print(f"❌ Error during analysis: {e}")
            return f"Analysis failed: {str(e)}"

    def generate_summary(self, analysis_results: str) -> str:
        """Generate a summary of the analysis results."""
        context = self._get_context(f"veterinary diagnosis summary {analysis_results}")

        try:
            summary = self.summary_chain.invoke(
                {"context": context, "analysis_results": analysis_results}
            )
            return summary
        except Exception as e:
            print(f"❌ Error generating summary: {e}")
            return f"Summary generation failed: {str(e)}"


class AiVetPipeline:
    """Complete pipeline for veterinary blood test analysis."""

    def __init__(
        self,
        vision_model_provider: str = "ollama",
        vision_model_name: str = "gemma3:4b",
        llm_provider: str = "ollama",
        llm_model_name: str = "qwen2.5:7b",
        knowledge_base_path: Optional[str] = None,
        **llm_kwargs,
    ):

        self.extractor = ImageToTextExtractor(
            vision_model_name=vision_model_name,
            vision_model_provider=vision_model_provider,
        )
        self.analyzer = VeterinaryTextAnalyzer(
            llm_provider, llm_model_name, knowledge_base_path, **llm_kwargs
        )
        print("🏥 AiVet Pipeline initialized successfully")

    def process_blood_test(
        self,
        file_path: str,
        analysis_mode: str = "few_shot",
        generate_summary: bool = True,
    ) -> Dict[str, str]:
        """Complete processing pipeline for blood test analysis."""

        results = {}

        try:
            # Step 1: Extract text from image/PDF
            print("🔍 Step 1: Extracting text from image...")
            extracted_text = self.extractor.extract_text_from_file(file_path)
            results["extracted_text"] = extracted_text

            # Step 2: Analyze blood results
            print("🧪 Step 2: Analyzing blood test results...")
            analysis = self.analyzer.analyze_blood_results(
                extracted_text, analysis_mode
            )
            results["analysis"] = analysis

            # Step 3: Generate summary (optional)
            if generate_summary:
                print("📋 Step 3: Generating health summary...")
                summary = self.analyzer.generate_summary(analysis)
                results["summary"] = summary

            print("✅ Pipeline completed successfully")
            return results

        except Exception as e:
            print(f"❌ Pipeline failed: {e}")
            results["error"] = str(e)
            return results
        finally:
            # Clean up GPU memory
            self.extractor.clear_memory()


class AiVetPipeline:
    """Complete pipeline for veterinary blood test analysis."""

    def __init__(
        self,
        vision_provider: str = "ollama",
        vision_model_name: str = "gemma3:4b",
        llm_provider: str = "ollama",
        llm_model_name: str = "qwen2.5:8b",
        knowledge_base_path: Optional[str] = None,
        vision_config: Optional[Dict] = None,
        **llm_kwargs,
    ):

        vision_config = vision_config or {}
        self.extractor = ImageToTextExtractor(
            vision_model_name=vision_model_name,
            vision_model_provider=vision_provider,
            **vision_config,
        )
        self.analyzer = VeterinaryTextAnalyzer(
            llm_provider, llm_model_name, knowledge_base_path, **llm_kwargs
        )
        print(
            f"🏥 AiVet Pipeline initialized with {vision_provider} vision and {llm_provider} LLM"
        )

    def process_blood_test(
        self,
        file_path: str,
        analysis_mode: str = "few_shot",
        generate_summary: bool = True,
    ) -> Dict[str, str]:
        """Complete processing pipeline for blood test analysis."""

        results = {}

        try:
            # Step 1: Extract text from image/PDF
            print("🔍 Step 1: Extracting text from image...")
            extracted_text = self.extractor.extract_text_from_file(file_path)
            results["extracted_text"] = extracted_text

            # Step 2: Analyze blood results
            print("🧪 Step 2: Analyzing blood test results...")
            analysis = self.analyzer.analyze_blood_results(
                extracted_text, analysis_mode
            )
            results["analysis"] = analysis

            # Step 3: Generate summary (optional)
            if generate_summary:
                print("📋 Step 3: Generating health summary...")
                summary = self.analyzer.generate_summary(analysis)
                results["summary"] = summary

            print("✅ Pipeline completed successfully")
            return results

        except Exception as e:
            print(f"❌ Pipeline failed: {e}")
            results["error"] = str(e)
            return results
        finally:
            # Clean up GPU memory
            self.extractor.clear_memory()

In [7]:
# pipeline_hf = AiVetPipeline(
#     vision_model_name="Qwen/Qwen2-VL-2B-Instruct",
#     vision_provider="huggingface",
#     llm_provider="qwen",
#     knowledge_base_path="./vet_knowledge_base/"
# )

# Example 2: Using Ollama Gemma 3 multimodal model
# pipeline_ollama = AiVetPipeline(
#     vision_model_name="qwen2.5vl:7b",  # or "gemma2:27b-vision" for larger model
#     vision_provider="ollama",
#     llm_provider="ollama",  # Using Ollama for text analysis too
#     llm_model_name="qwen2.5:7b",  # Using Ollama for text analysis too
#     knowledge_base_path="./vet_knowledge_base/",
#     vision_config={
#         "temperature": 0.1,
#         "top_k": 1,
#         "num_ctx": 32000,  # Take advantage of Gemma 3's 128K context
#     },
# )

# # Process a blood test image
# results = pipeline_ollama.process_blood_test(
#     file_path="../data/input/202402_hope_emo.pdf",
#     analysis_mode="few_shot",
#     generate_summary=True,
# )

# print("📊 Results:")
# for key, value in results.items():
#     print(f"\n{key.upper()}:\n{value}\n{'-'*50}")

In [8]:
context = """Here is the extracted information from the veterinary blood test report:

1. **RBC (Red Blood Cells)**
   - Detected Level: 7.18
   - Normal Range: 5.90 - 8.10 10^6/µl
   - Status: Normal

2. **HB (Hemoglobin)**
   - Detected Level: 15.6
   - Normal Range: 13.1 - 18.9 g/dl
   - Status: Normal

3. **HCT (Hematocrit)**
   - Detected Level: 44.6
   - Normal Range: 38.6 - 54.5 %
   - Status: Normal

4. **MCV (Mean Corpuscular Volume)**
   - Detected Level: 62.1
   - Normal Range: 61 - 72.6 fL
   - Status: Normal

5. **MCH (Mean Corpuscular Hemoglobin)**
   - Detected Level: 22.0
   - Normal Range: 20 - 26 pg
   - Status: Normal

6. **MCHC (Mean Corpuscular Hemoglobin Concentration)**
   - Detected Level: 35.00
   - Normal Range: 30 - 37 g/dl
   - Status: Normal

7. **CH (Cholesterol)**
   - Detected Level: 21.1
   - Normal Range: 21.0 - 26.5 mg/dl
   - Status: Normal

8. **CHDW (Cholesterol Distribution Width)**
   - Detected Level: 2.79
   - Normal Range: 1.7 - 3.80 mg/dl
   - Status: Normal

9. **HDW (Hemoglobin Distribution Width)**
   - Detected Level: 1.7
   - Normal Range: 1.60 - 2.27 g/dl
   - Status: Normal

10. **RDW (Red Cell Distribution Width)**
    - Detected Level: 13.8
    - Normal Range: 10.9 - 14.5 %
    - Status: Normal

11. **WBC (White Blood Cells)**
    - Detected Level: 13.88
    - Normal Range: 5.70 - 13.80 10^3/µl
    - Status: Normal

12. **Neutrophils (Neut)**
    - Detected Level: 11381.6
    - Normal Range: 3800 - 8800/µl
    - Status: Normal

13. **Lymphocytes (Linf)**
    - Detected Level: 1665.6
    - Normal Range: 1300 - 4100/µl
    - Status: Normal

14. **Monocytes (Mono)**
    - Detected Level: 694.0
    - Normal Range: 200 - 740/µl
    - Status: Normal

15. **Eosinophils (Eos)**
    - Detected Level: 138.8
    - Normal Range: 150 - 1100/µl
    - Status: Normal

16. **Basophils (Baso)**
    - Detected Level: 0
    - Normal Range: 0 - 100/µl
    - Status: Normal

17. **Platelets (PLT)**
    - Detected Level: 101
    - Normal Range: 150 - 460 1000/µl
    - Status: Low

18. **MPV (Mean Platelet Volume)**
    - Detected Level: 9.7
    - Normal Range: 8.6 - 18 fL
    - Status: Normal

19. **PCT (Platelet Crit)**
    - Detected Level: 0.10
    - Normal Range: 0.24 - 0.70
    - Status: Low

20. **MPC (Mean Platelet Count)**
    - Detected Level: 26.20
    - Normal Range: 53 - 70
    - Status: Normal

21. **MPD (Mean Platelet Distribution Width)**
    - Detected Level: 57.50
    - Normal Range: 53 - 70 %
    - Status: Normal

22. **LPLT (Low Platelet Level)**
    - Detected Level: 4
    - Normal Range: 1 - 50 1000/µl
    - Status: Low

23. **Plasma Opalescence**
    - Detected Level: Plasma opalescent
    - Status: Normal
--------------------------------------------------"""

In [9]:
import os
import shutil
import torch
from typing import List, Optional
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


class LocalVectorStore:
    """
    Encapsulates a persistent Chroma client and collection with consistent settings.
    """

    # Class-level settings to ensure consistency
    CHROMA_SETTINGS = Settings(
        allow_reset=True,
        anonymized_telemetry=False,
        is_persistent=True
    )

    def __init__(
        self, persist_path: str = "./chroma_db", collection_name: str = "vet_docs"
    ):
        self.persist_path = persist_path
        self.collection_name = collection_name
        self.client = None
        self.collection = None
        
        # Initialize with consistent settings
        self._initialize_client()

    def _initialize_client(self):
        """Initialize ChromaDB client with consistent settings."""
        try:
            # Always use the same settings for consistency
            self.client = chromadb.PersistentClient(
                path=self.persist_path, 
                settings=self.CHROMA_SETTINGS
            )
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name
            )
            print(f"✅ Successfully connected to ChromaDB at: {self.persist_path}")
            
        except Exception as e:
            if "readonly" in str(e).lower():
                print(f"⚠️ Database readonly error - likely settings mismatch: {e}")
                print("🔧 This usually means the DB was created with different settings")
                raise RuntimeError(
                    f"Database settings mismatch. Please delete {self.persist_path} "
                    "and run with overwrite=True to recreate with consistent settings."
                )
            else:
                raise

    def reset(self):
        """Properly reset the ChromaDB client and delete all data."""
        print("🔄 Resetting ChromaDB...")
        
        # Step 1: Properly close/reset the client if it exists
        if self.client is not None:
            try:
                # Try to delete the collection first
                if self.collection is not None:
                    try:
                        self.client.delete_collection(name=self.collection_name)
                        print(f"🗑️ Deleted collection: {self.collection_name}")
                    except Exception as e:
                        print(f"⚠️ Could not delete collection: {e}")
                
                # Reset the client if possible
                try:
                    self.client.reset()
                    print("🔄 Reset ChromaDB client")
                except Exception as e:
                    print(f"⚠️ Could not reset client: {e}")
                    
            except Exception as e:
                print(f"⚠️ Error during client cleanup: {e}")
            finally:
                # Clear references
                self.client = None
                self.collection = None
        
        # Step 2: Remove the entire directory
        if os.path.exists(self.persist_path):
            try:
                shutil.rmtree(self.persist_path)
                print(f"🗑️ Deleted DB directory: {self.persist_path}")
            except PermissionError as e:
                print(f"❌ Permission error: {e}")
                # Try to fix permissions and retry
                try:
                    for root, dirs, files in os.walk(self.persist_path):
                        for d in dirs:
                            os.chmod(os.path.join(root, d), 0o777)
                        for f in files:
                            os.chmod(os.path.join(root, f), 0o777)
                    shutil.rmtree(self.persist_path)
                    print(f"🗑️ Successfully deleted DB after permission fix")
                except Exception as e2:
                    raise RuntimeError(f"Cannot delete database directory: {e2}")
        
        # Step 3: Reinitialize with fresh client
        self._initialize_client()

    def add_documents(
        self,
        texts: List[str],
        embeddings: List[List[float]],
        metadatas: Optional[List[dict]] = None,
    ):
        """Add documents to the collection."""
        if not self.collection:
            raise RuntimeError("Collection not initialized")
            
        ids = [f"doc-{i}" for i in range(len(texts))]
        
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=metadatas or [{} for _ in texts],
            ids=ids,
        )

    def query(self, query_embedding: List[float], top_k: int = 5):
        """Query the collection."""
        if not self.collection:
            raise RuntimeError("Collection not initialized")
            
        return self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k,
        )


class VetVectorStoreBuilder:
    """Builds and manages a veterinary domain vector store with Chroma and HuggingFace embeddings."""

    def __init__(self, embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.embedding_model = embedding_model
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.embedding_model,
            model_kwargs={"device": device},
        )

    @staticmethod
    def sanitize_metadata(metadata: dict) -> dict:
        """Sanitize metadata to ensure ChromaDB compatibility."""
        simple_types = (str, int, float, bool, type(None))
        sanitized = {}
        for k, v in metadata.items():
            if isinstance(v, simple_types):
                sanitized[k] = v
            else:
                sanitized[k] = str(v)
        return sanitized

    def load_documents(self, docs_path: str) -> List:
        """Load documents from the specified path."""
        documents = []
        if os.path.isdir(docs_path):
            for filename in os.listdir(docs_path):
                filepath = os.path.join(docs_path, filename)
                try:
                    if filename.endswith(".pdf"):
                        loader = PyPDFLoader(filepath)
                        loaded_docs = loader.load()
                        if not loaded_docs or all(
                            not d.page_content.strip() for d in loaded_docs
                        ):
                            raise ValueError("Fallback to OCR")
                    elif filename.endswith(".txt"):
                        loader = TextLoader(filepath)
                        loaded_docs = loader.load()
                    else:
                        print(f"⚠️ Skipping unsupported file type: {filename}")
                        continue
                except Exception:
                    print(f"⚠️ Falling back to OCR for: {filepath}")
                    loader = UnstructuredPDFLoader(filepath, mode="elements")
                    loaded_docs = loader.load()

                print(f"📄 Loaded {len(loaded_docs)} docs from {filename}")
                documents.extend(loaded_docs)
        else:
            raise ValueError(f"Documents path is not a directory: {docs_path}")

        if not documents:
            raise ValueError("No documents loaded from the provided path")

        return documents

    def build_vectorstore(
        self, docs_path: str, persist_path: str = "./chroma_db", overwrite: bool = False
    ):
        """Build the vector store with improved error handling."""
        print(f"🚀 Starting vector store build...")
        print(f"📁 Documents path: {docs_path}")
        print(f"💾 Persist path: {persist_path}")
        print(f"🔄 Overwrite mode: {overwrite}")
        
        # If overwrite is True, delete the directory BEFORE creating the vector store
        if overwrite and os.path.exists(persist_path):
            print("🔄 Overwrite mode: deleting existing database...")
            shutil.rmtree(persist_path)
            print(f"🗑️ Deleted existing DB at: {persist_path}")
        
        documents = self.load_documents(docs_path)

        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = splitter.split_documents(documents)
        if not splits:
            splits = [doc for doc in documents if doc.page_content.strip()]

        texts = [doc.page_content for doc in splits]
        metadatas = [self.sanitize_metadata(doc.metadata) for doc in splits]

        print(f"🧠 Preparing to embed {len(texts)} text chunks...")

        # Initialize vector store AFTER potential deletion
        vector_store = LocalVectorStore(persist_path=persist_path)

        # Generate embeddings
        print("🔢 Generating embeddings...")
        embeddings = self.embeddings.embed_documents(texts)
        
        # Add documents to vector store
        print("💾 Adding documents to vector store...")
        vector_store.add_documents(texts, embeddings, metadatas=metadatas)

        print(f"✅ Vector store created at: {persist_path}")
        print(f"📊 Total documents indexed: {len(texts)}")

In [10]:
# import py7zr

# archive_path = "../vetdb/Veterinary Hematology, A Diagnostic Guide and Color Atlas (VetBooks.ir).7z"
# extract_path = "../vetdb"
# password = "vetbooks.ir"

# with py7zr.SevenZipFile(archive_path, mode='r', password=password) as archive:
#     archive.extractall(path=extract_path)

# print("Extraction complete!")

In [13]:
if os.path.exists("./chroma_db"):
    shutil.rmtree("./chroma_db")

In [14]:
builder = VetVectorStoreBuilder()
builder.build_vectorstore(
    docs_path="../vetdb",  # folder with PDFs/text
    persist_path="./chroma_db",  # where to store Chroma vectors
    overwrite=True,  # set False to append
)

🚀 Starting vector store build...
📁 Documents path: ../vetdb
💾 Persist path: ./chroma_db
🔄 Overwrite mode: True
⚠️ Falling back to OCR for: ../vetdb/Understanding-Blood-Results-A-Pet-Owners-Guide.pdf
📄 Loaded 80 docs from Understanding-Blood-Results-A-Pet-Owners-Guide.pdf
🧠 Preparing to embed 80 text chunks...
✅ Successfully connected to ChromaDB at: ./chroma_db
🔢 Generating embeddings...
💾 Adding documents to vector store...


InternalError: Query error: Database error: error returned from database: (code: 1032) attempt to write a readonly database

In [ ]:
# --- Simulated blood test result ---
blood_results = context
# --- Setup: With RAG ---
print("🔍 Testing with RAG enabled...")
analyzer_with_rag = VeterinaryTextAnalyzer(
    llm_provider="ollama",
    llm_model_name="qwen2.5:7b",
    knowledge_base_path="../vetdb",  # Path to your veterinary knowledge base (folder or file)
)

analysis_with_rag = analyzer_with_rag.analyze_blood_results(
    blood_results, mode="few_shot"
)
summary_with_rag = analyzer_with_rag.generate_summary(analysis_with_rag)

print("\n📊 Analysis with RAG:\n", analysis_with_rag)
print("\n📝 Summary with RAG:\n", summary_with_rag)

In [ ]:
# --- Setup: Without RAG ---
print("\n\n🔍 Testing WITHOUT RAG...")
analyzer_without_rag = VeterinaryTextAnalyzer(
    llm_provider="ollama",
    llm_model_name="qwen2.5:7b",
    knowledge_base_path=None,  # No RAG support
)

analysis_without_rag = analyzer_without_rag.analyze_blood_results(
    blood_results, mode="few_shot"
)
summary_without_rag = analyzer_without_rag.generate_summary(analysis_without_rag)

print("\n📊 Analysis without RAG:\n", analysis_without_rag)
print("\n📝 Summary without RAG:\n", summary_without_rag)